In [ ]:
import nest_asyncio
nest_asyncio.apply()

## ShellCommandTask

In addition to `FunctionTask`, pydra allows for creating tasks from shell commands by using `ShellCommandTask`.

Let's run a simple command `pwd` using pydra

In [ ]:
import pydra

In [ ]:
cmd = "pwd"
# we should use executable to pass the command we want to run
shelly = pydra.ShellCommandTask(name="shelly", executable=cmd)

# we can always check the cmdline of our task
shelly.cmdline

and now let's try to run it:

In [ ]:
with pydra.Submitter(plugin="cf") as sub:
    sub(shelly)


and check the result

In [ ]:
shelly.result()

the result should have `return_code`, `stdout` and `stderr`. If everything goes well `return_code` should be `0`, `stdout` should point to the working directory and `stderr` should be an empty string.

### Commands with arguments and inputs
you can also use longer command by providing a list:

In [ ]:
cmd = ["echo", "hail", "pydra"]
shelly = pydra.ShellCommandTask(name="shelly", executable=cmd)
print("cmndline = ", shelly.cmdline)

with pydra.Submitter(plugin="cf") as sub:
    sub(shelly)
shelly.result()

#### using args
In addition to `executable`, we can also use `args`. Last example can be also rewritten:

In [ ]:
cmd = "echo"
args = ["hail", "pydra"]

shelly = pydra.ShellCommandTask(name="shelly", executable=cmd, args=args)
print("cmndline = ", shelly.cmdline)

with pydra.Submitter(plugin="cf") as sub:
    sub(shelly)
shelly.result()

### Customized input

Pydra always checks `executable` and `args`, but we can also provide additional inputs, in order to do it, we have to modify `input_spec` first by using `SpecInfo` class:

In [ ]:
import attr

my_input_spec = pydra.specs.SpecInfo(
    name="Input",
    fields=[
        ("text", str, {"position": 1,
                       "argstr": "", 
                       "help_string": "text", 
                       "mandatory": True}),
    ],
    bases=(pydra.specs.ShellSpec,),
)

Notice, that in order to create your own `input_spec`, you have to provide a list of `fields`. There are several valid syntax to specify elements of `fields`. In the example above, the attribute has `type` and `metadata`, but the full specification can be found [here](https://www.attrs.org/en/stable/api.html#attr.ib). 

In `metadata`, you can provide additional information that is used by `pydra`, `help_string` is the only key that is required, and the full list of supported keys is `['position', 'argstr', 'requires', 'mandatory', 'allowed_values', 'output_field_name', 'copyfile', 'separate_ext', 'container_path', 'help_string', 'xor', 'output_file_template']`. Among the supported keys, you have:
- `help_string`: a sring, description of the argument;
- `position`: integer grater than 0, defines the relative position of the arguments when the shell command is constructed;
- `argstr`: a string, e.g. "-o", can be used to specify a flag if needed for the command argument; 
- `mandatory`: a bool, if True` pygra will raise an exception, if the argument is not provided; 

After defining `my_input_spec`, we can define our task:

In [ ]:
cmd_exec = "echo"
hello = "HELLO"
shelly = pydra.ShellCommandTask(
    name="shelly", executable=cmd_exec, text=hello, input_spec=my_input_spec
)

print("cmndline = ", shelly.cmdline)

with pydra.Submitter(plugin="cf") as sub:
    sub(shelly)
shelly.result()

### Customized output

We can also customized output if we want to return something more than the `stdout`, e.g. a file.

In [ ]:
my_output_spec = pydra.specs.SpecInfo(
    name="Output",
    fields=[("newfile", pydra.specs.File, "newfile_tmp.txt")],
    bases=(pydra.specs.ShellOutSpec,),
)


now we can create a task that returns a new file:

In [ ]:
cmd = ["touch", "newfile_tmp.txt"]
shelly = pydra.ShellCommandTask(name="shelly", executable=cmd, output_spec=my_output_spec)

print("cmndline = ", shelly.cmdline)

with pydra.Submitter(plugin="cf") as sub:
    sub(shelly)
shelly.result()

# DO NOT RUN IF Docker IS NOT AVAILABLE

**Note, that the following task use Docker, so they will fail if the Docker is not available. It will also fail in Binder.**

### DockerTask

all the commands can be also run in a docker container using `DockerTask`. Syntax is very similar, but additional argument `image` is required.

In [15]:
cmd = "whoami"
docky = pydra.DockerTask(name="docky", executable=cmd, image="busybox")

with pydra.Submitter() as sub:
    docky(submitter=sub)

docky.result()

Result(output=Output(return_code=0, stdout='root\n', stderr=''), runtime=None, errored=False)

#### Using `ShellCommandTask` with `container_info` argument:

You can run the shell command in a docker container by adding `container_info` argument to `ShellCommandTask`:

In [16]:
shelly = pydra.ShellCommandTask(name="shelly", executable="whoami", container_info=("docker", "busybox"))
with pydra.Submitter() as sub:
    shelly(submitter=sub)

shelly.result()

Result(output=Output(return_code=0, stdout='root\n', stderr=''), runtime=None, errored=False)

If we don't provide `container_info` the output should be different:

In [17]:
shelly = pydra.ShellCommandTask(name="shelly", executable="whoami")
with pydra.Submitter() as sub:
    shelly(submitter=sub)

shelly.result()

Result(output=Output(return_code=0, stdout='satra\n', stderr=''), runtime=None, errored=False)